In [1]:
# stdlib

# third party
import veilid

In [ ]:
NONCE_LENGTH = 24
QUIT = "QUIT"

In [ ]:
host = "localhost"
port = 5959

In [31]:
def connect(host: str, port: int):
    async def noop_callback(*args, **kwargs):
        # print("got callback", args, kwargs)
        return

    conn = await veilid.json_api_connect(host, port, noop_callback)
    return conn

IndentationError: expected an indented block after function definition on line 2 (1480544865.py, line 4)

In [32]:
# crypto_system = await conn.get_crypto_system(veilid.CryptoKind.CRYPTO_KIND_VLD0)
# async with crypto_system:
#     print("keygen:Generating a keypair")
#     my_keypair = await crypto_system.generate_key_pair()
#     print(f"keygen:Got {my_keypair=}")

# # await config.store_self_key(conn, my_keypair)

In [33]:
# stdlib

In [34]:
def load_keys():
    key_pair = "5MhuM8y3hcL4iAZspNQJJHCQyTVoA-DHVW-so2kmT20:6NweTbX2uFF3PfZgjjrYLr89TBU_eQ0WLu18JgnFwq8"
    my_keypair = veilid.KeyPair(key_pair)
    print("Own keypair:")
    print("    Public: ", my_keypair.key())
    print("    Private: ", my_keypair.secret())
    their_key = veilid.PublicKey("FQkHoYMuJSWhNl3bi2avFstA19QULCiQJtxx_hr2Shg")
    return my_keypair, their_key

In [35]:
my_keypair, their_key = load_keys()

Own keypair:
    Public:  5MhuM8y3hcL4iAZspNQJJHCQyTVoA-DHVW-so2kmT20
    Private:  6NweTbX2uFF3PfZgjjrYLr89TBU_eQ0WLu18JgnFwq8


In [36]:
their_key

'FQkHoYMuJSWhNl3bi2avFstA19QULCiQJtxx_hr2Shg'

In [51]:
async def create_sender(
    router: veilid.api.RoutingContext,
    crypto_system: veilid.CryptoSystem,
    key: veilid.TypedKey,
    secret: veilid.SharedSecret,
    send_subkey: veilid.ValueSubkey,
):
    """Read input and write it to the DHT."""

    async def encrypt(cleartext: str) -> bytes:
        """Encrypt the message with the shared secret and a random nonce."""

        print("encrypt:Getting a nonce")
        nonce = await crypto_system.random_nonce()
        print(f"encrypt:Received {nonce=}")
        print(f"encrypt:Encrypting {cleartext=}, {nonce=}, {secret=}")
        encrypted = await crypto_system.crypt_no_auth(cleartext.encode(), nonce, secret)
        ciphertext = nonce.to_bytes() + encrypted
        print(f"encrypt:{ciphertext=}")
        return ciphertext

    async def send(cleartext: str):
        """Write the encrypted version of the text to the DHT."""

        ciphertext = await encrypt(cleartext)
        print(f"send:Setting DHT {key=}, {send_subkey=}, to {ciphertext=}")
        await router.set_dht_value(key, send_subkey, ciphertext)

    # Prime the pumps. Especially when starting the conversation, this
    # causes the DHT key to propagate to the network.
    return send

In [17]:
async def receiver(
    router: veilid.api.RoutingContext,
    crypto_system: veilid.CryptoSystem,
    key: veilid.TypedKey,
    secret: veilid.SharedSecret,
    recv_subkey: veilid.ValueSubkey,
    name: str,
):
    """Wait for new data from the DHT and write it to the screen."""

    async def decrypt(payload: bytes) -> str:
        """Decrypt the payload with the shared secret and the payload's nonce."""

        print(f"decrypt:Decrypting {payload!r}")
        nonce = veilid.Nonce.from_bytes(payload[:NONCE_LENGTH])
        print(f"decrypt:Found {nonce=}")
        ciphertext = payload[NONCE_LENGTH:]
        print(f"decrypt:Decrypting {ciphertext=}, {nonce=}, {secret=}")
        cleartext = await crypto_system.crypt_no_auth(ciphertext, nonce, secret)
        print(f"decrypt:{cleartext=}")
        return cleartext.decode()

    last_seq = -1
    while True:
        # In the real world, don't do this. People may tease you for it.
        # This is meant to be easy to understand for demonstration
        # purposes, not a great pattern. Instead, you'd want to use the
        # callback function to handle events asynchronously.

        # Try to get an updated version of the receiving subkey.
        print(f"receiver:Getting the DHT value of {key=}, {recv_subkey=}")
        resp = await router.get_dht_value(key, recv_subkey, True)
        if resp is None:
            print("receiver:Didn't find a value")
            continue

        # If the other party hasn't sent a newer message, try again.
        if resp.seq == last_seq:
            print(f"receiver:DHT {key=} is still at {resp.seq=}")
            continue

        msg = await decrypt(resp.data)
        if msg == QUIT:
            print("receiver:Received the QUIT signal from the other end.")
            print("Other end closed the chat.")
            return

        print("receiver:Got new {msg=} at DHT {key=}, {resp.seq=}")
        print(f"\n{name}> {msg}")
        last_seq = resp.seq

In [30]:
conn = connect(host, port)

NameError: name 'connect' is not defined

In [38]:
def create_members(my_keypair, their_key):
    members = [
        veilid.DHTSchemaSMPLMember(my_keypair.key(), 1),
        veilid.DHTSchemaSMPLMember(their_key, 1),
    ]
    return members

In [ ]:
members = create_members()

In [41]:
router = await (await conn.new_routing_context()).with_default_safety()
crypto_system = await conn.get_crypto_system(veilid.CryptoKind.CRYPTO_KIND_VLD0)

name = "friendsname"

async with crypto_system, router:
    print("start:Getting a DH shared secret")
    secret = await crypto_system.cached_dh(their_key, my_keypair.secret())
    print(f"start:Got {secret=}")

start:Getting a DH shared secret
start:Got secret='ZcFA4LxhXis2ZPmxhvFnWHQHG1CROLVJTO0_bGgZqi4'


In [42]:
router = await (await conn.new_routing_context()).with_default_safety()
crypto_system = await conn.get_crypto_system(veilid.CryptoKind.CRYPTO_KIND_VLD0)

name = "friendsname"

async with crypto_system, router:
    print("start:Getting a DH shared secret")
    secret = await crypto_system.cached_dh(their_key, my_keypair.secret())
    print(f"start:Got {secret=}")

start:Getting a DH shared secret
start:Got secret='ZcFA4LxhXis2ZPmxhvFnWHQHG1CROLVJTO0_bGgZqi4'


In [43]:
router = await (await conn.new_routing_context()).with_default_safety()
crypto_system = await conn.get_crypto_system(veilid.CryptoKind.CRYPTO_KIND_VLD0)

name = "friendsname"

async with crypto_system, router:
    print("start:Creating a new DHT record")
    record = await router.create_dht_record(veilid.DHTSchema.smpl(0, members))
    print(f"New chat key: {record.key}")
    print("Give that to your friend!")

start:Creating a new DHT record
New chat key: VLD0:3_Bg783KyqMnHLAe22DSnbcJaNpt0R4WSGpLBi6LbwU
Give that to your friend!


In [47]:
record.key

'VLD0:MeQeEREO1mmzYRm7IZk73V5ia_NaSBVjkxAavpXam5c'

In [49]:
router = await (await conn.new_routing_context()).with_default_safety()
crypto_system = await conn.get_crypto_system(veilid.CryptoKind.CRYPTO_KIND_VLD0)

name = "friendsname"

async with crypto_system, router:
    print(f"start:Reopening the DHT record {record.key=} with {my_keypair=}")
    await router.close_dht_record(record.key)

start:Reopening the DHT record record.key='VLD0:MeQeEREO1mmzYRm7IZk73V5ia_NaSBVjkxAavpXam5c' with my_keypair='5MhuM8y3hcL4iAZspNQJJHCQyTVoA-DHVW-so2kmT20:6NweTbX2uFF3PfZgjjrYLr89TBU_eQ0WLu18JgnFwq8'


In [50]:
router = await (await conn.new_routing_context()).with_default_safety()
crypto_system = await conn.get_crypto_system(veilid.CryptoKind.CRYPTO_KIND_VLD0)

name = "friendsname"

async with crypto_system, router:
    print(f"start:Reopening the DHT record {record.key=} with {my_keypair=}")
    await router.open_dht_record(record.key, my_keypair)

start:Reopening the DHT record record.key='VLD0:MeQeEREO1mmzYRm7IZk73V5ia_NaSBVjkxAavpXam5c' with my_keypair='5MhuM8y3hcL4iAZspNQJJHCQyTVoA-DHVW-so2kmT20:6NweTbX2uFF3PfZgjjrYLr89TBU_eQ0WLu18JgnFwq8'


In [57]:
router = await (await conn.new_routing_context()).with_default_safety()
crypto_system = await conn.get_crypto_system(veilid.CryptoKind.CRYPTO_KIND_VLD0)

name = "friendsname"

async with crypto_system, router:
    # The party initiating the chat writes to subkey 0 and reads from subkey 1.
    # send_task = asyncio.create_task(sender(router, crypto_system, record.key, secret, 0))
    sender = await create_sender(router, crypto_system, record.key, secret, 0)
    await sender("testtesttest")

    # send = await send_task
    # await send("hello world")
    # recv_task = asyncio.create_task(receiver(router, crypto_system, record.key, secret, 1, name))

    # try:
    # print("start:Starting the chat")
    # await asyncio.wait([send_task, recv_task], return_when=asyncio.FIRST_COMPLETED)
    # finally:
    #     print(f"start:Closing the DHT record {record.key=}")
    #     await router.close_dht_record(record.key)
    #     print(f"start:Deleting the DHT record {record.key=}")
    #     await router.delete_dht_record(record.key)

    # print("start:Cleaning up")
    # recv_task.cancel()
    # send_task.cancel()

encrypt:Getting a nonce
encrypt:Received nonce='41RMKqkKv7umhqWnLMc68kaaoVqpXpO0'
encrypt:Encrypting cleartext='testtesttest', nonce='41RMKqkKv7umhqWnLMc68kaaoVqpXpO0', secret='ZcFA4LxhXis2ZPmxhvFnWHQHG1CROLVJTO0_bGgZqi4'
encrypt:ciphertext=b'\xe3TL*\xa9\n\xbf\xbb\xa6\x86\xa5\xa7,\xc7:\xf2F\x9a\xa1Z\xa9^\x93\xb4\x03\xad\xa0O\xe7\xe0\xdc\xd0\n\xc0\x97\x95'
send:Setting DHT key='VLD0:MeQeEREO1mmzYRm7IZk73V5ia_NaSBVjkxAavpXam5c', send_subkey=0, to ciphertext=b'\xe3TL*\xa9\n\xbf\xbb\xa6\x86\xa5\xa7,\xc7:\xf2F\x9a\xa1Z\xa9^\x93\xb4\x03\xad\xa0O\xe7\xe0\xdc\xd0\n\xc0\x97\x95'


In [22]:
record.key

'VLD0:hSkGLoVMIT6Mx2DIQ2-oghxPX2wfgTxObI1pizQB0C0'

In [ ]:
# await send("hello world2")

encrypt:Getting a nonce


ValueError: Response cs_op does not match request cs_op

In [ ]:
# stdlib
import time

In [ ]:
message = f"Hello from the world! {time.time()}"
message

In [ ]:
# await asyncio.create_task(sender(router, crypto_system, record.key, secret, 0, message))